## Analysis setup

We begin by loading the Distributions and LowRankApprox Julia packages, as well as some function definitions used in the code chunks below.

In [1]:
using Distributions
using LowRankApprox
include("../code/julia/datasim.jl");
include("../code/julia/likelihood.jl");
include("../code/julia/ash.jl");
include("../code/julia/mixSQP.jl");

Next, initialize the sequence of pseudorandom numbers.

In [2]:
srand(1);

## Generate a small data set

Let's start with a smaller example with 5,000 samples.

In [3]:
x = normtmixdatasim(5000);

## Compute the likelihood matrix

Compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals, with $k = 20$. Note that the rows of the likelihood matrix are normalized by default.

In [4]:
sd = autoselectmixsd(x, gridmult = 1.375);
L  = normlikmatrix(x,sd = sd);
size(L)

(5000, 20)

## Fit mixture model using SQP algorithm 

Observe that only a small number of iterations is needed to converge to the solution of the constrained optimization problem.

In [5]:
out = mixSQP(L, x = ones(size(L,2))/size(L,2), convtol = 1e-8,
             pqrtol = 1e-10, eps = 1e-8, sptol = 1e-3,
             maxiter = 100, maxqpiter = 100,
             lowrank = "svd", seed = 1, verbose = true);

Running SQP algorithm with the following settings:
- 5000 x 20 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial SVD tolerance  = 1.00e-10
- partial SVD max. error = 1.73e-09
iter       objective -min(g+1) #nnz #qp
   1 2.47539249e+03 +4.45e-01   20   0
   2 1.75265768e+03 +3.50e+04    1  30
   3 1.16160122e+03 +1.20e+04    1  14
   4 1.09130438e+03 +5.22e+03    3  43
   5 1.06636501e+03 +2.41e+03    3  28
   6 1.04509224e+03 +1.19e+03    3  19
   7 1.03423266e+03 +6.37e+02    3   7
   8 1.01931434e+03 +3.15e+02    2   8
   9 1.00587765e+03 +1.59e+02    3  21
  10 1.00148985e+03 +8.02e+01    3   7
  11 9.91249275e+02 +3.99e+01    3  16
  12 9.81087089e+02 +1.98e+01    3  22
  13 9.77327006e+02 +9.77e+00    4  24
  14 9.72127358e+02 +4.69e+00    3  29
  15 9.69514294e+02 +2.16e+00    4  12
  16 9.68516666e+02 +9.48e-01    4  13
  17 9.67920270e+02 +3.82e-01    4  10
  18 9.67489808e+02 +1.13e-01    4   7
  19 9.67238871e+02 +2.01e-02    5   9
  

## Generate a larger data set

Next, let's see what happens when we use the SQP algorithm to fit a mixture model to a much larger data set.

In [6]:
x = normtmixdatasim(100000);

## Compute the likelihood matrix

As before, we compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals. This time, we use a finer grid of $k = 100$ normal densities.

In [7]:
sd = autoselectmixsd(x, gridmult = 1.0685);
L  = normlikmatrix(x,sd = sd);
size(L)

(100000, 100)

## Fit mixture model using SQP algorithm 

Even on this much larger data set, only a small number of iterations is needed to compute the solution.

In [8]:
out = mixSQP(L, x = ones(size(L,2))/size(L,2), convtol = 1e-8,
             pqrtol = 1e-10, eps = 1e-8, sptol = 1e-3,
             maxiter = 100, maxqpiter = 100,
             lowrank = "qr", seed = 1, verbose = true);

Running SQP algorithm with the following settings:
- 100000 x 100 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial QR tolerance  = 1.00e-10
- partial QR max. error = 2.75e-08
iter       objective -min(g+1) #nnz #qp
   1 6.82359802e+04 +7.70e-01  100   0
   2 5.10425051e+04 +7.74e+04    1 100
   3 2.94656541e+04 +2.72e+04    1  54
   4 2.40476848e+04 +1.19e+04    2 100
   5 2.30013029e+04 +5.61e+03    2  57
   6 2.22589255e+04 +2.77e+03    2  63
   7 2.18716099e+04 +1.38e+03    2  84
   8 2.14881758e+04 +6.94e+02    3  71
   9 2.11402934e+04 +3.48e+02    3  76
  10 2.08215862e+04 +1.75e+02    2  46
  11 2.05391373e+04 +8.81e+01    3  45
  12 2.02560390e+04 +4.41e+01    3  40
  13 2.00388584e+04 +2.21e+01    3  34
  14 1.98259356e+04 +1.10e+01    3  34
  15 1.96780269e+04 +5.42e+00    3  33
  16 1.95582619e+04 +2.60e+00    3  48
  17 1.94819909e+04 +1.20e+00    4  44
  18 1.94389532e+04 +5.25e-01    4  44
  19 1.94129151e+04 +2.01e-01    4  50
 

## Session information

The section gives information about the computing environment used to generate the results contained in this
notebook, including the version of Julia, and the versions of the Julia packages used here. 

In [9]:
Pkg.status("Distributions");
Pkg.status("LowRankApprox");

 - Distributions                 0.15.0
 - LowRankApprox                 0.1.0


In [10]:
versioninfo()

Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-7567U CPU @ 3.50GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Prescott)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, broadwell)
